In [1]:
import sys
import time
import warnings
sys.path.append("..")
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns

from tma.object import Object
from tma.functions import get_df
from tma.model import Model
from tma.algorithms import swarm

import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

Создаем модель наблюдателя:

In [2]:
print("Время последнего моделирования:", datetime.now().strftime("%d/%m/%Y %H:%M\n"))

observer_x, observer_y, observer_course, observer_velocity = 0.0, 0.0, 0.0, 5.0
observer = Object(
    "Наблюдатель",
    observer_x,
    observer_y,
    observer_course,
    observer_velocity,
    verbose=True,
)

observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Время последнего моделирования: 10/04/2021 20:35
Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [3]:
# sns.set_style("white")
# for end_time in end_time_arr:
#     tma = TMA(observer, target, end_t=end_time, noise_std=np.radians(0.3))
#     dict_results = tma.swarm(n=500, p0=p0, seeded=True, fixed_target=True)
#     df = get_df(dict_results)
#     b_c = df.loc[1, 'Птек_ист']
#     d_c = df.loc[1, 'Дтек_ист']
#     sc1 = [b_c - 0.5, d_c * 0.85, 45 - 10, 10 * 0.9]
#     sc2 = [b_c + 0.5, d_c * 1.15, 45 + 10, 10 * 1.1]
#     xlim1 = [b_c - 0.6, d_c * 0.7, 45 - 15, 10 * 0.7]
#     xlim2 = [b_c + 0.6, d_c * 1.3, 45 + 15, 10 * 1.3]
#     fig, ax = plt.subplots(1, 4, figsize = (15, 3))
#     columns = ['Птек_расч', 'Дтек_расч', 'К0_расч', 'V0_расч']
#     for i, col in enumerate(columns):
#         ax[i].set(xlim=(xlim1[i], xlim2[i]))
#         sns.histplot(df[col], ax=ax[i], color="#496595");
#         ax[i].axvline(x=sc1[i], color="#3f4d63")
#         ax[i].axvline(x=sc2[i], color="#3f4d63")
#         ax[i].text(sc1[i], ax[i].get_ylim()[1] * 1.02, 'Рэф', rotation=30)
#         ax[i].text(sc2[i], ax[i].get_ylim()[1] * 1.02, 'Рэф', rotation=30)
#         ax[i].set_ylabel('Распеделение')
#         sns.despine()

#     print('Расчетное время = {}, Рэф = {}'.format(end_time, df['Успех'].apply(lambda x: x[2]).mean(axis=0)))
#     plt.show();

In [4]:
# start_t = 420
# tau = 2
# end_t = len(observer.coords[0]) - 1
# t = range(start_t, end_t + 1, tau)
# columns = ['П0_расч', 'Д0_расч', 'К0_расч', 'V0_расч']
# std_arr = [0.01, 0.1, 0.2, 0.3, 0.5, 1.]
# fig, ax = plt.subplots(6, 4, figsize=(25, 40))
# for i, std in enumerate(std_arr):
#     tma = TMA(observer, target, noise_std=np.radians(std), seed=1, tau=tau)
#     r = tma.dynamic_mle([0., 25., 7., 0.])
#     df = get_df(r)
#     df['П0_расч'] = df['П0_расч'].apply(lambda x: x - 360 if x > 180 else x)
#     v = [0., 20., 45., 10.]
#     for j, col in enumerate(columns):
#         sns.lineplot(x=t[:120], y=df[col][:120], ax=ax[i, j], size=0.2, legend=False);
#         sns.lineplot(x=t[120:270], y=df[col][120:270], ax=ax[i, j], size=0.2, legend=False);
#         sns.lineplot(x=t[270:], y=df[col][270:], ax=ax[i, j], size=0.2, legend=False);
#         ax[i, j].axhline(v[j], linestyle='--', linewidth=1.5)
#         ax[i, j].set_xlabel('Время')
#         ax[i, j].set_title('CКО = {}°'.format(std))
#         ax[i, j].ticklabel_format(useOffset=False)

In [5]:
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
def target_func(seed=None):
    rng = np.random.RandomState(seed)
    b = 0
    d = rng.uniform(5, 50)
    c = rng.uniform(0, 180)
    v = rng.uniform(3, 15)
    return [b, d, c, v]
result_df = pd.DataFrame(columns=['Algorithm', 'std', 'sr_0', 'sr_1', 'sr_2', 'sr_3', 'Ка', 'Nf', 'Nf_max', 'Iter', 'Iter_max', 't', 't_max'])
mean_columns = ['Ка', 'Nf', 'Iter', 't']
max_columns = ['Nf', 'Iter', 't']
Path("../tests/test4/").mkdir(parents=True, exist_ok=True)

In [6]:
%%time
model = Model(observer)
for std in std_arr:
    model.noise_std = np.radians(std)
    dict_results = swarm(
        model,
        algorithm_name="ММП в реальном времени",
        n=1000,
        target_func=target_func,
        p0=p0,
        seeded=True,
    )
    df = get_df(dict_results)
    for i in range(3):
        res = pd.DataFrame({'Algorithm': "ММП в реальном времени", 'std': [std]})
        dfi = df.iloc[i::3, :].copy()
        for j in range(4):
            res['sr_' + str(j)] = [dfi['Успех'].apply(lambda x: x[j]).mean(axis=0)]
        res[mean_columns] = dfi[mean_columns].mean(axis=0)
        res[['Nf_max', 'Iter_max', 't_max']] = dfi[max_columns].apply(max, axis=0)
        result_df = result_df.append(res)

Wall time: 11min 40s


In [7]:
result_df.reset_index(drop=True)
result_df['Ка'] = result_df['Ка'].apply(np.sqrt)
result_df.to_excel('../tests/test4/dynamic_results.xlsx', index=False)

In [8]:
result_df = pd.read_excel('../tests/test4/dynamic_results.xlsx')
display_df = result_df.copy()
precisions = {'Ка' : '{0:.2f}', 'Nf' : '{0:.0f}', 'Nf_max': '{0:.0f}',\
'Iter' : '{0:.0f}', 'Iter_max' : '{0:.0f}', 't' : '{0:.4f}', 't_max' : '{0:.4f}'}
for col, precision in precisions.items():
    display_df[col] = display_df[col].apply(lambda x: precision.format(x))
display_df.rename(columns={'Algorithm':'Алгоритм', 'std':'СКО', 'sr_0':'Рэф1', 'sr_1':'Рэф2', 'sr_2':'Рэф3', 'sr_3':'Рэф4', 'Ка':'sqrt(Ка)'}, inplace=True)

In [9]:
display_df

,Алгоритм,СКО,Рэф1,Рэф2,Рэф3,Рэф4,sqrt(Ка),Nf,Nf_max,Iter,Iter_max,t,t_max
0,ММП в реальном времени,0.0,0.982,0.996,0.996,0.999,0.00,14,32,12,24,0.0035,0.0418
1,ММП в реальном времени,0.0,1.000,1.000,1.000,1.000,0.00,3,11,2,10,0.0010,0.0539
2,ММП в реальном времени,0.0,1.000,1.000,1.000,1.000,0.00,4,9,3,8,0.0013,0.0389
3,ММП в реальном времени,0.1,0.235,0.362,0.366,0.422,0.10,14,32,12,24,0.0034,0.0428
4,ММП в реальном времени,0.1,0.631,0.803,0.805,0.854,0.10,10,13,9,12,0.0025,0.0270
5,ММП в реальном времени,0.1,0.931,0.980,0.980,0.991,0.10,7,13,6,12,0.0024,0.0216
6,ММП в реальном времени,0.2,0.127,0.233,0.238,0.276,0.20,14,32,12,24,0.0032,0.0231
7,ММП в реальном времени,0.2,0.439,0.632,0.636,0.707,0.20,10,21,9,18,0.0026,0.0255
8,ММП в реальном времени,0.2,0.857,0.931,0.931,0.960,0.20,8,34,7,23,0.0023,0.0079
9,ММП в реальном времени,0.3,0.094,0.161,0.168,0.214,0.30,15,32,13,24,0.0034,0.0536


In [10]:
import docx
doc = docx.Document()
t = doc.add_table(display_df.shape[0]+1, display_df.shape[1])
for j in range(display_df.shape[-1]):
    t.cell(0,j).text = display_df.columns[j]
for i in range(display_df.shape[0]):
    for j in range(display_df.shape[-1]):
        t.cell(i+1,j).text = str(display_df.values[i,j])
doc.save('../tests/test4/Сводная таблица.docx')